In [11]:
import sys
import pandas as pd
from pathlib import Path
from IPython.display import clear_output
import dask
from dask import dataframe as dd
import numpy as np
import tensorflow as tf
import bct


In [4]:
pconncsv_dir = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/pconns_working/pconn_csv')
subjIDs = []
allArrays = []

for pconncsv in pconncsv_dir.glob('*.csv'):
    clear_output(wait=True)
    subjIDs.append(pconncsv.name.split('_')[0])
    
    csv_in_sq = pd.read_csv(pconncsv, sep="\t", header=None).values
    csv_in_flat = csv_in_sq[np.tril_indices(453, k = -1)]
    
    if len(allArrays) == 0:
        allArrays = csv_in_flat
    else:
        allArrays = np.vstack((allArrays, csv_in_flat))
    print(pconncsv)
    


          0         1         2         3         4         5         6    \
0    7.254330  0.444225  0.062299  0.148028  0.353600  0.238232  0.135298   
1    0.444225  7.254330  0.073531  0.161731  0.547493  0.432555  0.408408   
2    0.062299  0.073531  7.254330  0.267943  0.060269  0.130199  0.107711   
3    0.148028  0.161731  0.267943  7.254330  0.026982  0.124635  0.124687   
4    0.353600  0.547493  0.060269  0.026982  7.254330  0.221280  0.232834   
..        ...       ...       ...       ...       ...       ...       ...   
449  0.435726  0.355052 -0.258158  0.039682  0.233644  0.154653  0.145882   
450  0.186331  0.347739 -0.198650  0.080555  0.211095  0.226074  0.270173   
451  0.028474  0.106182 -0.092361 -0.067544  0.147000  0.210798  0.263634   
452  0.234580  0.271293 -0.211699 -0.021960  0.209177  0.145914  0.257617   
453 -0.026265 -0.030294 -0.242950 -0.018625 -0.058590  0.128278  0.174115   

          7         8         9    ...       444       445       446  \
0  

In [86]:
print(csv_in_sq)

NameError: name 'csv_in_sq' is not defined

In [12]:
allArraysPath = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/allArrays.npy')
# np.save(allArraysPath, allArrays)

subjIDs_arrayPath = Path('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/subjIDs.npy')
# np.save(subjIDs_arrayPath, np.asarray(subjIDs))

In [14]:
allArrays = np.load(allArraysPath)
subjIDs = np.load(subjIDs_arrayPath)

In [15]:
c_data = pd.read_table('/fast_scratch/jdr/ABCD/ABCD_3165_Pipe/Data/abcd_ptsd01.txt', header=0, skiprows=[1])
c_data = c_data[c_data["eventname"] == "baseline_year_1_arm_1"]
c_data = c_data.drop(['collection_id', 'abcd_ptsd01_id', 'dataset_id', 
                      'src_subject_id', 'interview_date', 'eventname', 'collection_title'], axis=1)
c_data['subjectkey'] = 'sub-' + c_data['subjectkey'].str.replace('_', '')
c_data = c_data.drop(['ExposureTotal', 'IPVTotal'], axis=1, errors='ignore')
c_data['ExposureTotal'] = c_data.iloc[:, 3:19].sum(axis=1)
c_data['IPVTotal'] = c_data.iloc[:, 7:18].sum(axis=1)
c_data['ExposureAny'] = np.where(c_data['ExposureTotal'] == 0, 0, 1)
c_data['IPVAny'] = np.where(c_data['IPVTotal'] == 0, 0, 1)
c_data['Sex01'] = np.where(c_data['sex'] == 'M', 1, 0) # Must use numeric data as input (not M, F)

In [16]:
subjIDs_df = pd.DataFrame(subjIDs,columns=['subjectkey'])
df_comb = subjIDs_df.merge(c_data, how='left', on='subjectkey')
df_comb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7220 entries, 0 to 7219
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   subjectkey            7220 non-null   object 
 1   interview_age         7216 non-null   float64
 2   sex                   7216 non-null   object 
 3   ksads_ptsd_raw_754_p  7004 non-null   float64
 4   ksads_ptsd_raw_755_p  7004 non-null   float64
 5   ksads_ptsd_raw_756_p  7004 non-null   float64
 6   ksads_ptsd_raw_757_p  7004 non-null   float64
 7   ksads_ptsd_raw_758_p  7004 non-null   float64
 8   ksads_ptsd_raw_759_p  7004 non-null   float64
 9   ksads_ptsd_raw_760_p  7004 non-null   float64
 10  ksads_ptsd_raw_761_p  7004 non-null   float64
 11  ksads_ptsd_raw_762_p  7004 non-null   float64
 12  ksads_ptsd_raw_763_p  7004 non-null   float64
 13  ksads_ptsd_raw_764_p  7004 non-null   float64
 14  ksads_ptsd_raw_765_p  7004 non-null   float64
 15  ksads_ptsd_raw_766_p 

In [50]:
def fill_lower_diag(a):
    n = int(np.sqrt(len(a)*2))+1
    mask = np.tri(n, dtype=bool, k=-1) # or np.arange(n)[:,None] > np.arange(n)
    out = np.zeros((n,n),dtype=np.float32)
    out[mask] = a
    return out

In [85]:
b = allArrays[0]
c = fill_lower_diag(b)
d = pd.DataFrame(c)

d

,0,1,2,3,4,5,6,7,8,9,...,443,444,445,446,447,448,449,450,451,452
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.941717,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.001003,0.070971,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,-0.076196,0.017864,0.668761,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.607574,0.619047,0.306019,0.180991,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
448,0.548854,0.570774,-0.024705,0.102906,0.355275,0.441192,0.285596,-0.036766,-0.088085,0.122532,...,0.947861,0.015091,0.885490,0.674321,0.644029,0.000000,0.000000,0.000000,0.000000,0.0
449,0.508344,0.440654,-0.031624,0.090269,0.196692,0.329512,0.257096,-0.030472,-0.027147,0.188920,...,0.656445,-0.156946,0.902986,0.603249,0.824646,1.058720,0.000000,0.000000,0.000000,0.0
450,0.321641,0.302914,-0.015139,0.188883,0.201686,0.430852,0.369475,-0.032234,-0.143735,0.136429,...,0.747465,0.117698,0.492674,0.614153,0.437112,1.161300,0.896232,0.000000,0.000000,0.0
451,-0.047775,-0.076199,0.207355,0.343423,0.014560,0.223011,0.309602,0.017517,-0.174605,0.223427,...,0.171429,0.366268,0.115451,0.508634,0.189207,0.289918,0.178846,0.543393,0.000000,0.0


In [87]:
import csv

ROI_labels_path = Path('/fast_scratch/jdr/atlases/Schefer400Tian4/Schaefer2018_400Parcels_7Networks_order_Tian_Subcortex_S4_label_reformatted.txt')
with open(ROI_labels_path, newline='') as f:
    reader = csv.reader(f, delimiter='\t')
    ROI_labels = list(zip(*reader))[1]
    
e = d.rename(columns=ROI_labels)
print(e)

TypeError: 'tuple' object is not callable